In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
####Paquete para sistemas de recomendación surprise
###Puede generar problemas en instalación local de pyhton. Genera error instalando con pip
#### probar que les funcione para la próxima clase
!pip install scikit-surprise
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3156221 sha256=8192338604fde6c17ff111869805ef57adf01bcf41706fa98ae944e0b34b6547
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
conn=sql.connect('/content/db_movies')
cur=conn.cursor()

In [3]:
movies=pd.read_sql('select * from filtro_rat', conn )
movies.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [4]:
ratings=pd.read_sql('select * from filtro_rat', conn)
ratings=ratings.drop('timestamp',axis=1)
ratings.head(3)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0


In [5]:
####los datos deben ser leidos en un formato espacial para surprise
reader = Reader(rating_scale=(0.5, 5)) ### la escala de la calificación
###las columnas deben estar en orden estándar: user item rating
data   = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

In [6]:
#####Existen varios modelos
models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()]
results = {}

In [7]:
###knnBasiscs: calcula el rating ponderando por distancia con usuario/Items
###KnnWith means: en la ponderación se resta la media del rating, y al final se suma la media general
####KnnwithZscores: estandariza el rating restando media y dividiendo por desviación
####Knnbaseline: calculan el desvío de cada calificación con respecto al promedio y con base en esos calculan la ponderación

In [8]:
#### función para probar varios modelos ##########
model=models[1]
for model in models:

    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)

    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result


performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.683625,0.898307,0.213476,2.390731
knns.KNNWithZScore,0.689840,0.911445,0.208990,1.582961
knns.KNNWithMeans,0.695191,0.911969,0.228040,2.277895
knns.KNNBasic,0.739284,0.970415,0.108630,1.767007


In [9]:
###################se escoge el mejor knn withmeans#########################
param_grid = { 'sim_options' : {'name': ['msd','cosine'], \
                                'min_support': [5], \
                                'user_based': [False, True]}
             }


In [11]:
### se afina si es basado en usuario o basado en ítem

gridsearchKNNWithMeans = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=2)

gridsearchKNNWithMeans.fit(data)


gridsearchKNNWithMeans.best_params["rmse"]
gridsearchKNNWithMeans.best_score["rmse"]
gs_model=gridsearchKNNWithMeans.best_estimator['rmse'] ### mejor estimador de gridsearch

In [12]:
################# Entrenar con todos los datos y Realizar predicciones con el modelo afinado

trainset = data.build_full_trainset() ### esta función convierte todos los datos en entrnamiento, las funciones anteriores dividen  en entrenamiento y evaluación
model=gs_model.fit(trainset) ## se reentrena sobre todos los datos posibles (sin dividir)

predset = trainset.build_anti_testset() ### crea una tabla con todos los usuarios y los libros que no han leido
#### en la columna de rating pone el promedio de todos los rating, en caso de que no pueda calcularlo para un item-usuario
len(predset)

predictions = gs_model.test(predset) ### función muy pesada, hace las predicciones de rating para todos los libros que no hay leido un usuario
### la funcion test recibe un test set constriuido con build_test method, o el que genera crosvalidate

predictions_df = pd.DataFrame(predictions) ### esta tabla se puede llevar a una base donde estarán todas las predicciones
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique() ### promedio de ratings
predictions_df.sort_values(by='est',ascending=False)

####### la predicción se puede hacer para un libro puntual
model.predict(uid=269397, iid='0446353205',r_ui='') ### uid debía estar en número e isb en comillas

Computing the msd similarity matrix...
Done computing similarity matrix.


Prediction(uid=269397, iid='0446353205', r_ui='', est=3.6007100857435552, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [13]:
predictions_df.head()

,uid,iid,r_ui,est,details
0,1,318,3.60071,4.718386,"{'actual_k': 40, 'was_impossible': False}"
1,1,1704,3.60071,4.599829,"{'actual_k': 40, 'was_impossible': False}"
2,1,6874,3.60071,4.594140,"{'actual_k': 40, 'was_impossible': False}"
3,1,8798,3.60071,4.139595,"{'actual_k': 40, 'was_impossible': False}"
4,1,46970,3.60071,4.192381,"{'actual_k': 40, 'was_impossible': False}"


In [14]:
model.predict(uid=1, iid='46970',r_ui='') ### uid debía estar en número e isb en comilla

Prediction(uid=1, iid='46970', r_ui='', est=3.6007100857435552, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [15]:
def recomendaciones(user_id,n_recomend=10):

    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending = False).head(n_recomend)

    recomendados = predictions_userID[['iid','est']]
    recomendados.to_sql('reco',conn,if_exists="replace")

    recomendados=pd.read_sql('''select a.*, b.title
                             from reco a left join movies b
                             on a.iid=b.movieId ''', conn)

    return(recomendados)

In [16]:
recomendaciones(user_id=7,n_recomend=10)

,index,iid,est,title
0,30651,161131,5.0,War Dogs (2016)
1,30601,3461,5.0,Lord of the Flies (1963)
2,29178,161582,5.0,Hell or High Water (2016)
3,31814,3927,5.0,Fantastic Voyage (1966)
4,28473,93,5.0,Vampire in Brooklyn (1995)
5,30364,140715,5.0,Straight Outta Compton (2015)
6,30362,140174,5.0,Room (2015)
7,31387,135137,5.0,Pixels (2015)
8,29711,108156,5.0,Ride Along (2014)
9,28988,2184,5.0,"Trouble with Harry, The (1955)"
